In [ ]:
from fins3666.defines import *
from fins3666.utils import load_dataframe, load_market, format_market_data, current_fx_data
from fins3666.portfolio import Portfolio, Order


fx = load_market(os.path.join(DIR_DATA_RAW, 'fx-historical-daily.csv'))
ir = load_dataframe(os.path.join(DIR_DATA_RAW, 'ir-historical.csv'))

startTime = datetime(2001, 1, 1)

startingAssets = [
    {'asset': 'USD', 'units': ACCOUNT_SIZE_USD, 'unit_value_USD': 1.0},
    {'asset': 'AUD', 'units': 0, 'unit_value_USD': 0},
    {'asset': 'CAD', 'units': 0, 'unit_value_USD': 0},
    {'asset': 'CHF', 'units': 0, 'unit_value_USD': 0},
    {'asset': 'EUR', 'units': 0, 'unit_value_USD': 0},
    {'asset': 'GBP', 'units': 0, 'unit_value_USD': 0},
    {'asset': 'JPY', 'units': 0, 'unit_value_USD': 0},
    {'asset': 'NZD', 'units': 0, 'unit_value_USD': 0},
    {'asset': 'SEK', 'units': 0, 'unit_value_USD': 0},
    {'asset': 'SGD', 'units': 0, 'unit_value_USD': 0}]

SamplePortfolio = Portfolio(startTime, startingAssets)


specTime = datetime(2020, 12, 9)

formattedData = current_fx_data(fx, specTime)
# print('----------------------------------')
# for jawn in formattedData:
#     for k, v in jawn.items():
#         print(f'{k}: {v}')
#     print('----------------------------------')
dates = pd.date_range(start='2023-01-01', end='2023-01-03', freq='D')

for (idx, date) in enumerate(dates):
    currentData = current_fx_data(fx, date)
    SamplePortfolio.updateMarketData(currentData)
    testBuyOrder = Order(asset='AUD', units=1203.80, currency='USD',
                         order='buy', timestamp=date, order_type='market')
    testSellOrder = Order(asset='AUD', units=1203.80, currency='USD',
                          order='sell', timestamp=date, order_type='market')
    testBuyOrder1 = Order(asset='CHF', units=1203.80, currency='JPY',
                          order='buy', timestamp=date, order_type='market')
    testSellOrder1 = Order(asset='CHF', units=1203.80, currency='JPY',
                           order='sell', timestamp=date, order_type='market')
    SamplePortfolio.new_order(testBuyOrder)
    SamplePortfolio.new_order(testSellOrder)
    SamplePortfolio.new_order(testBuyOrder1)
    SamplePortfolio.new_order(testSellOrder1)

    SamplePortfolio.update(date)
    print(SamplePortfolio)

Executed market buy order of 1203.8 units of AUD for 0.6783USD per unit. Transaction cost: 816.54USD
Executed market sell order of 1203.8 units of AUD for 0.6780USD per unit. Transaction gain: 816.18USD
Executed market buy order of 1203.8 units of CHF for 142.6595JPY per unit. Transaction cost: 171733.46JPY
Executed market sell order of 1203.8 units of CHF for 142.5654JPY per unit. Transaction gain: 171620.2JPY


                               Portfolio Overview                               
Asset         Units        Unit Value    Asset Value       Position    Weight
0         12000000.00          $1.00 $12,000,000.00           Long   100.00%
1               0.00          $0.00           $nan           None      nan%
2               0.00          $0.00           $nan           None      nan%
3               0.00          $0.00           $nan           None      nan%
4               0.00          $0.00           $nan           None      nan%
5               0.00          $0.00        

In [3]:
specTime = '2020-12-09'

currentData = current_fx_data(fx, date)


test = '¥'

print(test)

"""
LEDGER FORMAT
    'OpenTimestamp': pd.Series(dtype='pd.datetime64[ns]'),
    'UpdateTimestamp': pd.Series(dtype='pd.datetime64[ns]'),
    'Asset': pd.Series(dtype='str'),
    'Units': pd.Series(dtype='float64'),
    'UnitWAP': pd.Series(dtype='float64'),
    'TransactionIdxs': pd.Series(dtype='list'),
    'AssetType': pd.Series(dtype='str'),
    'CloseTimestamp': pd.Series(dtype='pd.datetime64[ns]'),
    'Open': pd.Series(dtype='Bool')})

TRADE FORMAT

    'Timestamp': pd.Series(dtype='pd.datetime64[ns]'),
    'Asset': pd.Series(dtype='str'),
    'Units': pd.Series(dtype='float64'),
    'UnitPriceUSD': pd.Series(dtype='float64'),
    'OrderValueUSD': pd.Series(dtype='float64'),
    'ForecastReturn': pd.Series(dtype='float64'),
    'i_USD': pd.Series(dtype='float64'),
    'OrderType': pd.Series(dtype='str'),
    'Exchange': pd.Series(dtype='str'),
    'AssetType': pd.Series(dtype='str')})
"""

¥


"\nLEDGER FORMAT\n    'OpenTimestamp': pd.Series(dtype='pd.datetime64[ns]'),\n    'UpdateTimestamp': pd.Series(dtype='pd.datetime64[ns]'),\n    'Asset': pd.Series(dtype='str'),\n    'Units': pd.Series(dtype='float64'),\n    'UnitWAP': pd.Series(dtype='float64'),\n    'TransactionIdxs': pd.Series(dtype='list'),\n    'AssetType': pd.Series(dtype='str'),\n    'CloseTimestamp': pd.Series(dtype='pd.datetime64[ns]'),\n    'Open': pd.Series(dtype='Bool')})\n\nTRADE FORMAT\n\n    'Timestamp': pd.Series(dtype='pd.datetime64[ns]'),\n    'Asset': pd.Series(dtype='str'),\n    'Units': pd.Series(dtype='float64'),\n    'UnitPriceUSD': pd.Series(dtype='float64'),\n    'OrderValueUSD': pd.Series(dtype='float64'),\n    'ForecastReturn': pd.Series(dtype='float64'),\n    'i_USD': pd.Series(dtype='float64'),\n    'OrderType': pd.Series(dtype='str'),\n    'Exchange': pd.Series(dtype='str'),\n    'AssetType': pd.Series(dtype='str')})\n"